<a href="https://colab.research.google.com/github/HamzaKamranKhawaja/ai-internship/blob/main/UNIQUIFY_ASSIGNMENT_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras



In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)), 
    keras.layers.Dense(128, activation='relu'), 
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(24, activation="relu"), 
    keras.layers.Dense(10)
])

In [ ]:
#optimizers and metrics

optimizer = tf.keras.optimizers.Adam()
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')





In [ ]:
@tf.function
def train_step(images, labels): 
  with tf.GradientTape() as tape: 
    predictions = model(images, training=True)
    loss = loss_obj(labels, predictions)
  all_variables = model.trainable_variables 
  gradients = tape.gradient(loss, all_variables)
  optimizer.apply_gradients(zip(gradients, all_variables))
  
  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels): 
  predictions = model(images, training=False)
  t_loss = loss_obj(labels, predictions)
  
  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
#Process the data
mnist_data = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist_data.load_data()
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

#batch and shuffle
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).shuffle(10000).batch(32)


In [ ]:
#Train and test the model
EPOCHS = 7

for epoch in range(EPOCHS): 
  train_loss.reset_states
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds: 
    train_step(images, labels)
  
  for test_images, test_labels in test_ds: 
    test_step(test_images, test_labels)
  
  print(
      f'Epoch {epoch + 1} '
      f'Loss: {train_loss.result()}, '
      f'Accuracy: {train_accuracy.result() * 100} '
      f'Test Loss: {test_loss.result()}, '
      f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1 Loss: 1.1330633163452148, Accuracy: 80.74166870117188 Test Loss: 0.39653459191322327, Test Accuracy: 90.62999725341797
Epoch 2 Loss: 0.71539705991745, Accuracy: 92.39166259765625 Test Loss: 0.2520633637905121, Test Accuracy: 93.91999816894531
Epoch 3 Loss: 0.5469228029251099, Accuracy: 94.51166534423828 Test Loss: 0.20463420450687408, Test Accuracy: 94.4800033569336
Epoch 4 Loss: 0.45090872049331665, Accuracy: 95.61332702636719 Test Loss: 0.18119460344314575, Test Accuracy: 95.25
Epoch 5 Loss: 0.3863939642906189, Accuracy: 96.32666778564453 Test Loss: 0.13367028534412384, Test Accuracy: 96.30000305175781
Epoch 6 Loss: 0.339649498462677, Accuracy: 97.02333068847656 Test Loss: 0.1658947765827179, Test Accuracy: 95.51000213623047
Epoch 7 Loss: 0.30437836050987244, Accuracy: 97.4366683959961 Test Loss: 0.11609009653329849, Test Accuracy: 97.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
model.save_weights('/content/drive/MyDrive/Colab Notebooks/weights/a1-2')


Mounted at /content/drive
